# Time Series Analysis: "The Final Project"

`End? No, the journey doesn't end here. Death is just another path. One that we all must take.
-J.R.R. Tolkien, The Return of the King`

---

## Libraries

In [1]:
%cd "Documents\tsa2021-m5"

C:\Users\aamorado\Documents\tsa2021-m5


In [2]:
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
import statsmodels.api as sm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from pandas.plotting import register_matplotlib_converters
from IPython.display import display
from tsa_tools import *  # Hehe

register_matplotlib_converters()
sns.set_style('darkgrid')

np.set_printoptions(precision=4)
pd.set_option('precision', 4)


---

## M5 Forecasting

For this "final project", we will be forecasting the <b><u>level 9</b></u> series (unit sales of all products, aggregated for each store and department).

Load `sales_train_evaluation.csv` and use observations from `d_1 to d_1913` for training and `d_1914 to d_1941` for testing.

In [3]:
df_calendar = pd.read_csv('../data/m5/calendar.csv')
df_sales = pd.read_csv('../data/m5/sales_train_evaluation.csv')
df_weights = pd.read_csv('../data/m5/weights_validation.csv')
# display(df_calendar, df_sales, df_weights)

In [4]:
train_df = (df_sales.set_index([*df_sales.columns[5::-1]]).T
           .set_index(pd.DatetimeIndex(df_calendar.date)[:1941]).iloc[:-28])
test_df = (df_sales.set_index([*df_sales.columns[5::-1]]).T
           .set_index(pd.DatetimeIndex(df_calendar.date)[:1941]).iloc[-28:])
# display(train_df, test_df)

In [5]:
levels = {
    1: None,
    2: "state_id",
    3: "store_id",
    4: "cat_id",
    5: "dept_id",
    6: ["state_id", "cat_id"],
    7: ["state_id", "dept_id"],
    8: ["store_id", "cat_id"],
    9: ["store_id", "dept_id"],
    10: "item_id",
    11: ["state_id", "item_id"],
    12: ["store_id", "item_id"]
}


---

## Part 1. Baseline Methods (10 pts.)

### Q1. (10 pts.)

Extract all level 9 series from the dataset.

For each series, generate a 28-step forecast using the methods enumerated below and calculate the `RMSSE` against the test set:

1. `Naive`


2. `Seasonal Naive`


3. `SES`


4. `Holt's Linear`


5. `Additive Holt-Winters`

Summarize the metrics in a dataframe and print it.

In [6]:
methods = {
    "Naive": BaseFuncModel(naivef),
    "Seasonal Naive": BaseFuncModel(snaivef, m=7),
    "SES": StatsModelsWrapper(ETSModel, trend=None, seasonal=None),
    "Holt's Linear": StatsModelsWrapper(ETSModel, trend='add', seasonal=None),
    "Additive Holt-Winters": StatsModelsWrapper(
        ETSModel, seasonal_periods=7, trend='add', seasonal='add'),
}

trainOG_df_9 = train_df.sum(axis='columns', level=levels[9])
train_df_9 = timeSeriesFiltering(trainOG_df_9, lower=10)
test_df_9 = test_df.sum(axis='columns', level=levels[9])
weights_df_9 = (df_weights
                .loc[df_weights['Level_id'] == 'Level9']
                .set_index(['Agg_Level_1', 'Agg_Level_2'])[['Weight']])

In [7]:
res = {}
for method, model in methods.items():
    forecast_df_9 = pd.DataFrame(
        {label: model.fit(content).forecast(28)
        for label, content in train_df_9.items()}
        )
    res[method] = rateMyForecast(
        trainOG_df_9, test_df_9, forecast_df_9)['RMSSE']

In [8]:
pd.set_option('display.max_rows', None)
df_res_9_base = pd.DataFrame(res)
df_res_9_base.index = pd.MultiIndex.from_tuples(df_res_9_base.index)
df_res_9_base

Naive  Seasonal Naive     SES  Holt's Linear  \
CA_1 HOBBIES_1    1.4216          0.7428  0.8598         0.8589   
     HOBBIES_2    1.8601          1.0992  0.8494         0.8414   
     HOUSEHOLD_1  2.0500          0.5036  1.1212         1.1331   
     HOUSEHOLD_2  2.2407          0.5094  1.1894         1.1486   
     FOODS_1      0.8768          0.6880  0.8637         0.8808   
     FOODS_2      2.0036          0.8068  2.0035         2.2574   
     FOODS_3      1.6443          0.4750  1.0401         0.9787   
CA_2 HOBBIES_1    1.1753          0.6880  1.0901         1.0902   
     HOBBIES_2    1.3073          1.4373  1.3402         1.3308   
     HOUSEHOLD_1  1.9863          0.6240  1.3668         1.3564   
     HOUSEHOLD_2  1.8849          0.5770  1.4254         1.4165   
     FOODS_1      1.1962          1.0775  1.3218         1.3134   
     FOODS_2      4.1379          1.4366  2.2206         2.1603   
     FOODS_3      2.4898          0.5828  1.3979         1.3605   
CA_3 HOBBIES_1    0.9058          0.6425  0.6766         0.6784   
     HOBBIES_2    1.1844          1.6422  1.5348         1.5252   
     HOUSEHOLD_1  2.4874          0.7176  1.0695         1.0945   
     HOUSEHOLD_2  2.0081          0.8406  1.3608         1.3524   
     FOODS_1      1.0944          1.4881  1.3092         1.3022   
     FOODS_2      1.8145          0.7055  1.6655         1.6808   
     FOODS_3      1.4642          0.4057  0.8303         1.4798   
CA_4 HOBBIES_1    1.4108          1.0669  0.9649         0.9597   
     HOBBIES_2    1.5277          1.3413  1.3553         1.3549   
     HOUSEHOLD_1  3.2265          1.3949  1.1781         1.1518   
     HOUSEHOLD_2  1.2957          0.9160  1.0072         0.9944   
     FOODS_1      0.9428          1.4526  1.2850         1.2664   
     FOODS_2      1.2565          0.7737  1.1230         1.1156   
     FOODS_3      1.3082          0.5906  0.8758         0.8723   
TX_1 HOBBIES_1    1.5773          0.8856  0.8751         0.8737   
     HOBBIES_2    1.7232          1.5083  1.3088         1.2963   
     HOUSEHOLD_1  1.7292          0.7703  1.0973         1.0909   
     HOUSEHOLD_2  1.1629          1.2091  1.0126         0.9971   
     FOODS_1      1.2691          1.6348  1.4803         1.4703   
     FOODS_2      1.1590          0.9268  0.8335         0.8334   
     FOODS_3      1.1032          0.7950  0.8655         0.8366   
TX_2 HOBBIES_1    0.7631          0.8409  0.8406         0.8361   
     HOBBIES_2    0.7312          1.2235  0.9492         0.9444   
     HOUSEHOLD_1  0.8174          0.8049  0.8195         0.8222   
     HOUSEHOLD_2  1.2981          0.6212  0.8821         0.8852   
     FOODS_1      1.3761          1.1937  1.2033         1.1958   
     FOODS_2      0.9110          0.6606  0.7281         0.7461   
     FOODS_3      0.7848          0.5939  0.7153         0.7135   
TX_3 HOBBIES_1    1.4710          0.8672  0.9036         0.9004   
     HOBBIES_2    1.4024          1.6834  1.0940         1.0908   
     HOUSEHOLD_1  1.0028          1.0079  0.9570         0.9398   
     HOUSEHOLD_2  0.8211          1.2536  0.9665         0.9437   
     FOODS_1      2.5226          2.0495  1.9882         1.9710   
     FOODS_2      1.3254          1.3751  1.3809         1.3830   
     FOODS_3      0.9046          1.0331  0.9999         0.9814   
WI_1 HOBBIES_1    1.3352          0.4960  0.7349         0.7350   
     HOBBIES_2    1.1800          1.4764  1.1532         1.1458   
     HOUSEHOLD_1  1.6758          0.6321  1.0699         1.0707   
     HOUSEHOLD_2  1.6721          0.7258  1.0645         1.0628   
     FOODS_1      0.9442          0.8110  1.0012         0.9976   
     FOODS_2      2.0424          1.2113  1.5852         1.5796   
     FOODS_3      1.5857          0.5180  1.1000         1.0915   
WI_2 HOBBIES_1    1.3340          0.8068  0.7468         0.7363   
     HOBBIES_2    1.1058          1.4020  1.1380         1.1333   
     HOUSEHOLD_1  1.2729          1.0639  1.3340         1.2667   

---

## Part 2. LightGBM (30 pts.)

### Q2. (10 pts.)

For all series, use an un-tuned `LightGBM` with 56-day lookback that uses a one-step recursive forecasting strategy to generate a 28-step forecast.

Calculate the `RMSSE` against the test set, then summarize the metrics in a dataframe and print it.

In [9]:
model = RecursiveRegressor(
    lgb.LGBMRegressor(random_state=1, w=56, h=28, n_jobs=-1))  # Model: recursive-forecasting
pred = {}

for col in train_df_9:
    model.fit(None, train_df_9[col])
    pred[col] = model.predict(trainOG_df_9[col].iloc[-56:]).squeeze()

In [10]:
df_pred_9_rrlgb = pd.DataFrame(pred)
df_pred_9_rrlgb.index=test_df_9.index

df_res_9_rrlgb = rateMyForecast(
    trainOG_df_9, test_df_9, df_pred_9_rrlgb)['RMSSE']
res["RecursiveRegressor(LGBMRegressor)"] = df_res_9_rrlgb

df_res_9_rrlgb.index = pd.MultiIndex.from_tuples(df_res_9_rrlgb.index)
df_res_9_rrlgb.to_frame()

RMSSE
CA_1 HOBBIES_1    0.7180
     HOBBIES_2    0.6586
     HOUSEHOLD_1  0.5073
     HOUSEHOLD_2  0.5062
     FOODS_1      0.6677
     FOODS_2      0.6450
     FOODS_3      0.4652
CA_2 HOBBIES_1    0.6993
     HOBBIES_2    1.2472
     HOUSEHOLD_1  0.6587
     HOUSEHOLD_2  0.7485
     FOODS_1      0.9528
     FOODS_2      1.3704
     FOODS_3      0.5608
CA_3 HOBBIES_1    0.5075
     HOBBIES_2    1.3769
     HOUSEHOLD_1  0.6552
     HOUSEHOLD_2  0.7711
     FOODS_1      1.1578
     FOODS_2      0.5410
     FOODS_3      0.4244
CA_4 HOBBIES_1    1.0293
     HOBBIES_2    1.0370
     HOUSEHOLD_1  1.1065
     HOUSEHOLD_2  1.1063
     FOODS_1      1.4001
     FOODS_2      0.7879
     FOODS_3      0.6548
TX_1 HOBBIES_1    0.7179
     HOBBIES_2    1.0838
     HOUSEHOLD_1  0.6982
     HOUSEHOLD_2  0.8985
     FOODS_1      1.4479
     FOODS_2      0.7213
     FOODS_3      0.7457
TX_2 HOBBIES_1    0.7694
     HOBBIES_2    0.9201
     HOUSEHOLD_1  0.6974
     HOUSEHOLD_2  0.4701
     FOODS_1      1.0516
     FOODS_2      0.5892
     FOODS_3      0.5175
TX_3 HOBBIES_1    0.7456
     HOBBIES_2    1.1799
     HOUSEHOLD_1  0.7238
     HOUSEHOLD_2  1.0219
     FOODS_1      1.6606
     FOODS_2      1.0628
     FOODS_3      0.8803
WI_1 HOBBIES_1    0.4849
     HOBBIES_2    1.0659
     HOUSEHOLD_1  0.5406
     HOUSEHOLD_2  0.7215
     FOODS_1      0.7831
     FOODS_2      1.0008
     FOODS_3      0.5749
WI_2 HOBBIES_1    0.7607
     HOBBIES_2    1.2752
     HOUSEHOLD_1  0.9675
     HOUSEHOLD_2  1.0303
     FOODS_1      2.0849
     FOODS_2      1.4842
     FOODS_3      0.8791
WI_3 HOBBIES_1    0.8043
     HOBBIES_2    1.0026
     HOUSEHOLD_1  0.6797
     HOUSEHOLD_2  0.8251
     FOODS_1      1.4774
     FOODS_2      0.9445
     FOODS_3      0.5415

### Q3. (10 pts.)

For all series, use an un-tuned `LightGBM` with 56-day lookback that uses a direct forecasting strategy to generate a 28-step forecast.

Calculate the `RMSSE` against the test set, then summarize the metrics in a dataframe and print it.

In [11]:
model = MultiOutputRegressor(
    lgb.LGBMRegressor(random_state=1, n_jobs=-1),
    n_jobs=-1)  # Model: direct-forecasting
pred = {}

for col in train_df_9:
    X_train, _, y_train, _ = TimeseriesGenerator(
        X=train_df_9[col],
        y=None,
        w=56,
        h=28)
    model.fit(X_train, y_train)
    pred[col] = model.predict([trainOG_df_9[col].iloc[-56:]]).squeeze()

In [12]:
df_pred_9_morlgb = pd.DataFrame(pred, index=test_df_9.index)

df_res_9_morlgb = rateMyForecast(
    trainOG_df_9, test_df_9, df_pred_9_morlgb)['RMSSE']
res["MultiOutputRegressor(LGBMRegressor)"] = df_res_9_morlgb

df_res_9_morlgb.index = pd.MultiIndex.from_tuples(df_res_9_morlgb.index)
df_res_9_morlgb.to_frame()

RMSSE
CA_1 HOBBIES_1    0.6870
     HOBBIES_2    0.6902
     HOUSEHOLD_1  0.3635
     HOUSEHOLD_2  0.6683
     FOODS_1      0.6356
     FOODS_2      0.7556
     FOODS_3      0.5491
CA_2 HOBBIES_1    0.6626
     HOBBIES_2    1.4831
     HOUSEHOLD_1  0.5651
     HOUSEHOLD_2  0.8564
     FOODS_1      1.2147
     FOODS_2      1.5736
     FOODS_3      0.6949
CA_3 HOBBIES_1    0.5409
     HOBBIES_2    1.3213
     HOUSEHOLD_1  0.5652
     HOUSEHOLD_2  1.0248
     FOODS_1      1.1526
     FOODS_2      0.5205
     FOODS_3      0.4762
CA_4 HOBBIES_1    1.0659
     HOBBIES_2    1.0646
     HOUSEHOLD_1  1.1893
     HOUSEHOLD_2  1.2187
     FOODS_1      1.4446
     FOODS_2      0.8464
     FOODS_3      0.7849
TX_1 HOBBIES_1    0.7365
     HOBBIES_2    1.2643
     HOUSEHOLD_1  0.7898
     HOUSEHOLD_2  1.1766
     FOODS_1      1.4104
     FOODS_2      0.6312
     FOODS_3      0.7245
TX_2 HOBBIES_1    0.7980
     HOBBIES_2    1.0457
     HOUSEHOLD_1  0.6691
     HOUSEHOLD_2  0.4887
     FOODS_1      0.9875
     FOODS_2      0.6193
     FOODS_3      0.5281
TX_3 HOBBIES_1    0.6781
     HOBBIES_2    1.3581
     HOUSEHOLD_1  0.8329
     HOUSEHOLD_2  1.1091
     FOODS_1      1.5831
     FOODS_2      1.0183
     FOODS_3      0.9081
WI_1 HOBBIES_1    0.4860
     HOBBIES_2    1.0000
     HOUSEHOLD_1  0.5658
     HOUSEHOLD_2  0.7214
     FOODS_1      0.9579
     FOODS_2      0.9230
     FOODS_3      0.6724
WI_2 HOBBIES_1    0.7070
     HOBBIES_2    1.1984
     HOUSEHOLD_1  0.9822
     HOUSEHOLD_2  0.8728
     FOODS_1      2.4593
     FOODS_2      1.4717
     FOODS_3      0.8308
WI_3 HOBBIES_1    0.8515
     HOBBIES_2    1.0450
     HOUSEHOLD_1  0.6969
     HOUSEHOLD_2  0.8210
     FOODS_1      1.6087
     FOODS_2      0.8097
     FOODS_3      0.5013

### Q4. (10 pts.)

For all series, generate a 28-step forecast by combining the forecasts generated by the models in Q2 and Q3 (i.e. simple averaging).

Calculate the `RMSSE` against the test set, then summarize the metrics in a dataframe and print it.

In [13]:
df_pred_9_combo = (df_pred_9_morlgb + df_pred_9_rrlgb) / 2

df_res_9_combo = rateMyForecast(
    trainOG_df_9, test_df_9, df_pred_9_combo)['RMSSE']
res["Combo(LGBMRegressor)"] = df_res_9_combo

df_res_9_combo.index = pd.MultiIndex.from_tuples(df_res_9_combo.index)
df_res_9_combo.to_frame()

RMSSE
CA_1 HOBBIES_1    0.6904
     HOBBIES_2    0.6439
     HOUSEHOLD_1  0.4231
     HOUSEHOLD_2  0.5778
     FOODS_1      0.6198
     FOODS_2      0.6892
     FOODS_3      0.5021
CA_2 HOBBIES_1    0.6709
     HOBBIES_2    1.3542
     HOUSEHOLD_1  0.5993
     HOUSEHOLD_2  0.7949
     FOODS_1      1.0747
     FOODS_2      1.4515
     FOODS_3      0.5664
CA_3 HOBBIES_1    0.5164
     HOBBIES_2    1.3264
     HOUSEHOLD_1  0.5857
     HOUSEHOLD_2  0.8892
     FOODS_1      1.1380
     FOODS_2      0.5180
     FOODS_3      0.4338
CA_4 HOBBIES_1    1.0430
     HOBBIES_2    1.0373
     HOUSEHOLD_1  1.1338
     HOUSEHOLD_2  1.1502
     FOODS_1      1.4136
     FOODS_2      0.8051
     FOODS_3      0.6727
TX_1 HOBBIES_1    0.7211
     HOBBIES_2    1.1384
     HOUSEHOLD_1  0.7319
     HOUSEHOLD_2  1.0014
     FOODS_1      1.4205
     FOODS_2      0.6690
     FOODS_3      0.7282
TX_2 HOBBIES_1    0.7763
     HOBBIES_2    0.9732
     HOUSEHOLD_1  0.6745
     HOUSEHOLD_2  0.4608
     FOODS_1      1.0106
     FOODS_2      0.5957
     FOODS_3      0.4999
TX_3 HOBBIES_1    0.7028
     HOBBIES_2    1.2539
     HOUSEHOLD_1  0.7582
     HOUSEHOLD_2  1.0566
     FOODS_1      1.6066
     FOODS_2      1.0269
     FOODS_3      0.8766
WI_1 HOBBIES_1    0.4745
     HOBBIES_2    1.0194
     HOUSEHOLD_1  0.5475
     HOUSEHOLD_2  0.7089
     FOODS_1      0.8584
     FOODS_2      0.9439
     FOODS_3      0.6151
WI_2 HOBBIES_1    0.7195
     HOBBIES_2    1.2200
     HOUSEHOLD_1  0.9392
     HOUSEHOLD_2  0.9303
     FOODS_1      2.2685
     FOODS_2      1.4192
     FOODS_3      0.8371
WI_3 HOBBIES_1    0.8222
     HOBBIES_2    1.0168
     HOUSEHOLD_1  0.6759
     HOUSEHOLD_2  0.8082
     FOODS_1      1.5345
     FOODS_2      0.8282
     FOODS_3      0.4994

---

## Part 3. WRMSSE (10 pts.)

### Q5.  (10 pts.)

Calculate the `WRMSSE` for the all the methods described above. The weights can be found in `weights_validation.csv`.

For reference, the M5 benchmarks have the following `WRMSSE` scores at level 9:

- `Naive` = <b>1.764</b>


- `S.Naive` = <b>0.888</b>


- `ES_bu` = <b>0.728</b>

<i>Note: The M5 benchmarks use a bottom-up method for forecasting, so they will not necessarily be equal to your scores.</i>

In [14]:
df_res_9_all = pd.DataFrame(res)
df_res_9_all.index = pd.MultiIndex.from_tuples(df_res_9_all.index)
df_res_9_all

(df_res_9_all.rename_axis(['Agg_Level_1', 'Agg_Level_2'])
 .multiply(weights_df_9.squeeze(), axis=0).sum())

Naive                                  1.5616
Seasonal Naive                         0.8919
SES                                    1.1585
Holt's Linear                          1.1855
Additive Holt-Winters                  0.8200
RecursiveRegressor(LGBMRegressor)      0.7569
MultiOutputRegressor(LGBMRegressor)    0.7814
Combo(LGBMRegressor)                   0.7505
dtype: float64

---

## Part 4. Middle-Out Method (30 pts.)

### Q6. Bottom-Up (15 pts.)

Using your forecasts from the best performing method in Q5, use the bottom-up method described in [FPP3](https://otexts.com/fpp3/single-level.html) to generate forecasts for levels 1 to 8.

Calculate the `WRMSSE` for levels 1 to 8 against the test set, then summarize the metrics in a dataframe and print it.

For reference, you can find the benchmark `WRMSSE` scores in the `The M5 Accuracy competition: Results, findings and conclusions` paper.

<i>Note: The M5 benchmarks use a bottom-up method for forecasting, so they will not necessarily be equal to your scores.</i>

In [15]:
df_pred_9_combo.columns = train_df.sum(axis=1, level=["store_id", "state_id", "cat_id", "dept_id"]).columns
df_pred_9_combo

store_id        CA_1                                                        \
state_id          CA                                                         
cat_id       HOBBIES             HOUSEHOLD                 FOODS             
dept_id    HOBBIES_1 HOBBIES_2 HOUSEHOLD_1 HOUSEHOLD_2   FOODS_1   FOODS_2   
date                                                                         
2016-04-25  449.4956   42.4433    754.6167    195.0511  271.6638  541.5019   
2016-04-26  396.0684   37.1498    643.8261    177.7038  253.1075  435.8280   
2016-04-27  388.4789   39.5788    586.1311    184.2913  289.1053  366.2773   
2016-04-28  419.4912   37.9877    622.6572    184.2326  303.6351  340.3313   
2016-04-29  456.5489   47.5803    739.6181    215.0809  314.9435  460.0979   
2016-04-30  582.3975   55.4284   1019.7767    271.6790  372.1390  567.8636   
2016-05-01  546.6977   51.3810   1079.9215    278.5060  332.1003  623.2916   
2016-05-02  465.7320   35.9209    778.7456    195.6691  272.2665  537.8936   
2016-05-03  431.2596   42.9203    655.8035    185.2495  289.3279  496.3385   
2016-05-04  444.1696   41.6432    628.0217    192.5918  268.0387  497.2088   
2016-05-05  420.7036   41.4662    656.6979    205.8809  301.2285  471.3906   
2016-05-06  533.6297   45.8810    780.5747    233.7416  356.2364  623.3794   
2016-05-07  594.5917   51.2122   1025.3846    290.4712  424.2229  758.9690   
2016-05-08  560.6139   45.6659    972.4979    272.3124  367.4398  730.2405   
2016-05-09  479.0045   45.5947    739.1785    194.5277  297.9190  573.5590   
2016-05-10  462.7437   42.5986    632.5744    187.0334  278.1853  484.8036   
2016-05-11  394.9171   43.3479    619.6288    176.3259  331.4810  442.8389   
2016-05-12  417.1410   45.2155    631.6421    198.7958  328.5310  403.7696   
2016-05-13  457.1407   44.1961    750.4216    214.8100  364.7362  481.4330   
2016-05-14  577.9533   56.6859   1045.6635    291.6818  450.4492  599.6835   
2016-05-15  540.6957   49.9867   1059.2575    291.7307  380.3799  672.0483   
2016-05-16  499.2434   40.5267    792.7556    196.3841  352.4208  507.2847   
2016-05-17  418.9400   34.0043    651.1030    181.5334  350.4556  442.5061   
2016-05-18  423.5643   36.5153    608.5951    187.0856  338.2723  413.0950   
2016-05-19  465.6402   38.4391    629.9176    197.6384  338.5922  377.7564   
2016-05-20  451.4362   46.8942    761.8990    212.7949  368.7699  473.4516   
2016-05-21  586.8037   54.5120   1018.7414    287.9025  413.9527  551.9996   
2016-05-22  544.8824   59.5364   1108.8442    294.2332  391.8593  602.5771   

store_id                   CA_2                        ...      WI_2  \
state_id                     CA                        ...        WI   
cat_id                  HOBBIES             HOUSEHOLD  ...     FOODS   
dept_id       FOODS_3 HOBBIES_1 HOBBIES_2 HOUSEHOLD_1  ...   FOODS_1   
date                                                   ...             
2016-04-25  1917.5483  324.1858   42.1745    655.6467  ...  344.8301   
2016-04-26  1820.1153  335.1625   41.0074    602.7929  ...  327.2772   
2016-04-27  1732.6518  285.6967   42.3390    641.6630  ...  310.4772   
2016-04-28  1714.0109  338.5550   37.8039    631.6716  ...  266.4066   
2016-04-29  2244.1102  365.7659   38.3668    824.5203  ...  343.4292   
2016-04-30  2693.1875  462.4425   44.5834   1011.6265  ...  382.6277   
2016-05-01  2890.6448  457.0832   45.4321   1033.0461  ...  364.8620   
2016-05-02  2030.6053  309.9696   32.2123    650.4048  ...  365.2623   
2016-05-03  1866.7531  296.4192   34.6295    607.2990  ...  342.9247   
2016-05-04  1825.3116  312.1186   48.9701    635.2823  ...  335.2363   
2016-05-05  1825.1248  329.3737   35.8061    601.8712  ...  322.0492   
2016-05-06  2216.3633  414.7067   41.9109    830.4388  ...  341.0748   
2016-05-07  2786.3660  483.4882   47.9626   1095.6236  ...  387.8887   
2016-05-08  2979.9687  428.3690   42.9706   1097.3906  ...  359.4870   
2016-05-09  2108.7399  270.5431   35.2556    637.0947  ..

### Q7. Top-Down  (15 pts.)

Using your forecasts from the best performing method in Q5, use the top-down method with `average historical proportions` described in [FPP3](https://otexts.com/fpp3/single-level.html) to generate forecasts for levels 10 to 12.

Calculate the `WRMSSE` for levels 10 to 12  against the test set, then summarize the metrics in a dataframe and print it.

For reference, you can find the benchmark `WRMSSE` scores in the `The M5 Accuracy competition: Results, findings and conclusions` paper.

<i>Note: The M5 benchmarks use a bottom-up method for forecasting, so they will not necessarily be equal to your scores.</i>

In [16]:
# Your code here

# Time Series Analysis: "The Final Project"

`End? No, the journey doesn't end here. Death is just another path. One that we all must take.
-J.R.R. Tolkien, The Return of the King`

---

## Libraries

In [17]:
%cd "Documents\tsa2021-m5"

[WinError 3] The system cannot find the path specified: 'Documents\\tsa2021-m5'
C:\Users\aamorado\Documents\tsa2021-m5


In [18]:
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
import statsmodels.api as sm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from pandas.plotting import register_matplotlib_converters
from IPython.display import display
from tsa_tools import *  # Hehe

register_matplotlib_converters()
sns.set_style('darkgrid')

np.set_printoptions(precision=4)
pd.set_option('precision', 4)

---

## M5 Forecasting

For this "final project", we will be forecasting the <b><u>level 9</b></u> series (unit sales of all products, aggregated for each store and department).

Load `sales_train_evaluation.csv` and use observations from `d_1 to d_1913` for training and `d_1914 to d_1941` for testing.

In [19]:
df_calendar = pd.read_csv('../data/m5/calendar.csv')
df_sales = pd.read_csv('../data/m5/sales_train_evaluation.csv')
df_weights = pd.read_csv('../data/m5/weights_validation.csv')
# display(df_calendar, df_sales, df_weights)

In [20]:
train_df = (df_sales.set_index([*df_sales.columns[5::-1]]).T
           .set_index(pd.DatetimeIndex(df_calendar.date)[:1941]).iloc[:-28])
test_df = (df_sales.set_index([*df_sales.columns[5::-1]]).T
           .set_index(pd.DatetimeIndex(df_calendar.date)[:1941]).iloc[-28:])
# display(train_df, test_df)

In [21]:
levels = {
    1: None,
    2: "state_id",
    3: "store_id",
    4: "cat_id",
    5: "dept_id",
    6: ["state_id", "cat_id"],
    7: ["state_id", "dept_id"],
    8: ["store_id", "cat_id"],
    9: ["store_id", "dept_id"],
    10: "item_id",
    11: ["state_id", "item_id"],
    12: ["store_id", "item_id"]
}

In [22]:
train_df_9 = timeSeriesFiltering(
    train_df.sum(axis='columns', level=levels[9]), lower=10)
trainOG_df_9 = train_df.sum(axis='columns', level=levels[9])
test_df_9 = test_df.sum(axis='columns', level=levels[9])
weights_df_9 = (df_weights
                .loc[df_weights['Level_id'] == 'Level9']
                .set_index(['Agg_Level_1', 'Agg_Level_2'])[['Weight']])

---

## Part 5. King of the Hill (20 pts.)

Using whatever methods/models you desire, beat the best `WRMSSE` score in Q5.

<b><u>Do not tune your model using the test set!</b></u> If you do, you will not get points for this part.

### Q8. (10 pts.)

Describe your methodology here. 

Points will be awarded for <b>aesthetics</b> (ex. use of diagrams), <b>ease of reading</b>, <b>clarity</b>, and <b>brevity</b>.

Points will be deducted for <b>excessively long</b> walls of text and descriptions.

ANSWER HERE.

### Q9. (10 pts.)

This part is for your actual code.

In [23]:
X_train, _, y_train, _ = TimeseriesGenerator(
    X=train_df_9.iloc[:, 1], y=None, w=56, h=28)

In [24]:
X_train.shape, y_train.shape

((1830, 56), (1830, 28))

In [25]:
transformer = EndogenousTransformer(w=56, h=28, return_X=False, reshape=True)
y_transform = transformer.fit_transform(train_df_9.iloc[:, 1])
y_transform.shape

(1913,)
(1913,)
(1830, 28)


(1830, 28)

In [26]:
np.allclose(y_transform, y_train)

True

In [27]:
np.arange(4).reshape(-1, 1)

array([[0],
       [1],
       [2],
       [3]])

In [28]:
np.array(
    [
        [1],
        [2],
        [4]
    ]
).reshape(-1)

array([1, 2, 4])

In [29]:
train_df_9.iloc[:, 1].shape

(1913,)

In [30]:
regressor = MultiOutputRegressor(
        lgb.LGBMRegressor(random_state=1, n_jobs=-1),
        n_jobs=-1
    )
regressor.fit(X_train, y_train)
regressor.predict([trainOG_df_9.iloc[-56:1]])

ValueError: Found array with dim 3. Estimator expected <= 2.

In [ ]:
model = TransformedTargetRegressor(
    regressor = regressor,
    transformer = transformer,
    check_inverse = False
)

model.fit(X_train, train_df_9.iloc[:, 1])

(1913,)
(1913,)
(1830, 28)


TransformedTargetRegressor(check_inverse=False,
                           regressor=MultiOutputRegressor(estimator=LGBMRegressor(random_state=1),
                                                          n_jobs=-1),
                           transformer=EndogenousTransformer(h=28, reshape=True,
                                                             return_X=False,
                                                             w=56))

In [ ]:
model.predict([trainOG_df_9.iloc[-56:1]])

ValueError: Found array with dim 3. Estimator expected <= 2.

* gridsearch (light GMB onnly per TS)
* modelseach (per TS, baseline seasonal) + direct [singleshot] (treees)
* wrapper search (for GBM, recursive/direct[multipleshot]/chain)
* output is 70
[
    T-3[S1, S2...],
    T-2[S1, S2...],
    T-1[S1, S2...]
]

[S1, S2...]

* exog (price, holiday, DoW, DoY, WoM, etc)